In [1]:
from pathlib import Path 
import datetime
import os, dotenv
dotenv.load_dotenv()
os.chdir(Path(os.getenv("PYTHONPATH")).expanduser())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [3]:
VARIANT = "hybrid"
preformat_data_df = pd.read_json(f"data/training_datasets/train_test_split/all_{VARIANT}.jsonl", lines=True)
preformat_data_df['filetype'] = preformat_data_df['filetype'].apply(lambda x: 'cryptol' if x == 'cry' else 'saw')
preformat_data_df = preformat_data_df[preformat_data_df["filetype"] != "txt"].reset_index(drop=True)
preformat_data_df.head()


,filename,filetype,content,variant,set
0,cryptol/examples/splitAt.cry,cryptol,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",hybrid,unsupervised
1,cryptol/examples/AE.cry,cryptol,/*\nImplementation of the algorithms from the ...,hybrid,unsupervised
2,cryptol/examples/xor_cipher.cry,cryptol,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,hybrid,supervised
3,cryptol/examples/zero_weird.cry,cryptol,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,hybrid,unsupervised
4,cryptol/examples/builtin_lifting.cry,cryptol,"//builtins lift over tuples, seqs, and records...",hybrid,unsupervised


In [4]:
from src.preprocessing.sft_instruct_preprocess import iter_call_openai_structured, alpaca_df_to_qwen_messages
file_cache_path = f"cache/alpaca_instruct_cache/SFT_{VARIANT}_source_code.jsonl"

test_df = preformat_data_df.sample(1, random_state=42).reset_index(drop=True)
input_mode="full"



result = iter_call_openai_structured(preformat_data_df, "gpt-4.1-mini", input_mode, file_cache_path)

result = alpaca_df_to_qwen_messages(
    result, 
    output="content",
    #system_prompt="You are a meticulous assistant that writes formal specifications and verification code for Cryptol programs.", 
    drop_empty_output=False, 
    include_filename_in_user=False
    )


In [5]:
result.to_json(f"data/training_datasets/SFT_message_format_{VARIANT}_source_code.jsonl", orient="records", lines=True)